In [1]:
!pip install pandas numpy faker openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Initialize Faker
fake = Faker()

# Financial and Social Media Data
finance_products = ['Credit Card', 'Personal Loan', 'Mortgage', 'Investment Plan', 'Savings Account']
social_media_platforms = ['Twitter', 'Instagram', 'Facebook', 'LinkedIn', 'Reddit']

# Number of users
num_users = 5000

# Function to generate random transaction history
def generate_transaction_history():
    history = []
    num_transactions = random.randint(5, 20)
    for _ in range(num_transactions):
        category = random.choice(finance_products)
        amount = random.uniform(100, 20000)
        date = fake.date_between(start_date='-2y', end_date='today')
        history.append((category, amount, date))
    return history

# Function to generate social media activity
def generate_social_media_activity():
    platform = random.choice(social_media_platforms)
    post_frequency = random.randint(0, 50)  # Posts per week
    engagement_level = random.uniform(0, 500)  # Avg. likes/comments per post
    financial_mentions = random.choice([0, 1])  # 1 = user discusses finance-related topics

    return platform, post_frequency, engagement_level, financial_mentions

# Create dataset
data = []
for _ in range(num_users):
    user_id = fake.uuid4()
    age = random.randint(18, 70)
    gender = random.choice(['Male', 'Female'])
    income = random.uniform(20000, 150000)
    engagement = random.choice(['High', 'Medium', 'Low'])
    risk_profile = random.choice(['Low', 'Medium', 'High'])
    sentiment_score = random.uniform(-1, 1)
    transaction_history = generate_transaction_history()
    social_media = generate_social_media_activity()

    data.append([user_id, age, gender, income, engagement, risk_profile, sentiment_score, transaction_history] + list(social_media))

# Convert to DataFrame
df = pd.DataFrame(data, columns=['user_id', 'age', 'gender', 'income', 'engagement', 'risk_profile', 'sentiment_score', 'transaction_history',
                                 'platform_preference', 'post_frequency', 'engagement_level', 'financial_mentions'])

# Feature Extraction: Transactions
def extract_transaction_features(transaction_history):
    category_counts = {cat: 0 for cat in finance_products}
    total_spend = 0

    for trans in transaction_history:
        category, amount, _ = trans
        category_counts[category] += 1
        total_spend += amount

    return category_counts, total_spend

df['category_counts'], df['total_spend'] = zip(*df['transaction_history'].apply(extract_transaction_features))

# Convert category counts to DataFrame and merge
category_df = pd.DataFrame(df['category_counts'].to_list(), columns=finance_products)
df = pd.concat([df, category_df], axis=1)
df.drop(columns=['category_counts', 'transaction_history'], inplace=True)

# Sentiment-Based Recommendation
def get_sentiment_recommendation(sentiment_score):
    if sentiment_score > 0.5:
        return "Consider an Investment Plan to grow your wealth."
    elif sentiment_score < -0.5:
        return "A Savings Account might help you manage financial stress."
    else:
        return "A balanced Credit Card with rewards could be ideal."

df['sentiment_recommendation'] = df['sentiment_score'].apply(get_sentiment_recommendation)

# Clustering Users
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['age', 'income', 'sentiment_score', 'total_spend', 'post_frequency', 'engagement_level']])
kmeans = KMeans(n_clusters=5, random_state=42)
df['user_cluster'] = kmeans.fit_predict(scaled_features)

# Generate Cluster-Based Recommendation
def generate_cluster_recommendation(user_cluster):
    cluster_users = df[df['user_cluster'] == user_cluster]
    recommended_category = cluster_users[finance_products].sum().idxmax()
    return f"Users in your cluster, active on {random.choice(social_media_platforms)}, prefer {recommended_category}."

df['cluster_recommendation'] = df['user_cluster'].apply(generate_cluster_recommendation)

# Multi-Armed Bandit Class
class MultiArmedBandit:
    def __init__(self, arms):
        self.arms = arms
        self.successes = {arm: 1 for arm in arms}  # Start with small positive counts
        self.failures = {arm: 1 for arm in arms}

    def recommend(self):
        """ Recommend an arm (financial product) using Thompson Sampling. """
        sampled_means = {arm: np.random.beta(self.successes[arm], self.failures[arm]) for arm in self.arms}
        return max(sampled_means, key=sampled_means.get)

    def update(self, arm, reward):
        """ Update success and failure counts based on user response. """
        if reward:
            self.successes[arm] += 1  # User accepted
        else:
            self.failures[arm] += 1  # User ignored/rejected

# Initialize Bandit
bandit = MultiArmedBandit(finance_products)

# Simulated User Acceptance of Recommendation
def user_accepts_recommendation(recommended_product):
    acceptance_prob = {
        'Credit Card': 0.4,
        'Personal Loan': 0.3,
        'Mortgage': 0.2,
        'Investment Plan': 0.5,
        'Savings Account': 0.6
    }
    return random.random() < acceptance_prob[recommended_product]

# Generate Adaptive Recommendation & Update Multi-Armed Bandit
def get_adaptive_recommendation(user_cluster):
    recommended_product = bandit.recommend()
    reward = user_accepts_recommendation(recommended_product)

    # Social Media Weighting: More engagement = Higher Learning Weight
    user_engagement = df[df['user_cluster'] == user_cluster]['engagement_level'].mean()
    weight = 1 if user_engagement > 250 else 0  # Only update if engagement is high

    bandit.update(recommended_product, reward and weight)
    return recommended_product

df['adaptive_recommendation'] = df['user_cluster'].apply(get_adaptive_recommendation)

# Display Sample Recommendations
df[['user_id', 'platform_preference', 'sentiment_recommendation', 'cluster_recommendation', 'adaptive_recommendation']].head(10)


,user_id,platform_preference,sentiment_recommendation,cluster_recommendation,adaptive_recommendation
0,47f76f17-97ec-4556-a6e2-e40820c54e64,Instagram,A balanced Credit Card with rewards could be i...,"Users in your cluster, active on Instagram, pr...",Savings Account
1,42d31c6a-c5ea-493d-8e8c-f266093c88e4,LinkedIn,A Savings Account might help you manage financ...,"Users in your cluster, active on Instagram, pr...",Investment Plan
2,1fcbb05b-6468-414a-854a-453356a1040c,Twitter,A Savings Account might help you manage financ...,"Users in your cluster, active on Reddit, prefe...",Mortgage
3,a383375d-2a14-4d95-91ab-0f0e794b17e1,LinkedIn,Consider an Investment Plan to grow your wealth.,"Users in your cluster, active on Facebook, pre...",Credit Card
4,b1e8d6c7-07d5-42d2-b6f7-23f9f7446d6f,Reddit,Consider an Investment Plan to grow your wealth.,"Users in your cluster, active on Facebook, pre...",Savings Account
5,337db968-506a-4af6-9b02-526049bc3858,Instagram,Consider an Investment Plan to grow your wealth.,"Users in your cluster, active on Reddit, prefe...",Personal Loan
6,3c00411c-1652-45bc-ad1c-4f5218cade1f,Reddit,A balanced Credit Card with rewards could be i...,"Users in your cluster, active on Instagram, pr...",Personal Loan
7,f608f47c-8885-45ab-b4f3-c8528b4601d4,Instagram,A balanced Credit Card with rewards could be i...,"Users in your cluster, active on Facebook, pre...",Investment Plan
8,b92d45c5-74d1-477d-812b-04ec1754d97c,LinkedIn,A balanced Credit Card with rewards could be i...,"Users in your cluster, active on Reddit, prefe...",Mortgage
9,35092867-33c2-4242-aacc-7f9d17fa6546,Reddit,A balanced Credit Card with rewards could be i...,"Users in your cluster, active on Facebook, pre...",Investment Plan
